In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import time
import os
import copy
import json

from tqdm import tqdm
from sklearn.metrics import average_precision_score, precision_recall_curve

from torch.utils.data import Dataset
import skimage
from PIL import Image
from copy import deepcopy
from sklearn.utils.fixes import signature
import time
import pickle
import nltk

plt.ion()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

np.set_printoptions(precision=2, suppress=True)

In [2]:
class AnswerDisDataset(Dataset):
    def __init__(self, dataset_name, split):
        super(AnswerDisDataset, self).__init__()
        
        self.image_dir = None
        if dataset_name == 'vqa_2.0':
            self.image_dir = '/home/qing/Desktop/Datasets/MSCOCO/images'
        elif dataset_name == 'vizwiz':
            self.image_dir = '/home/qing/Desktop/Datasets/VizWiz/v1/data/Images'
        self.image_ext = '.jpg'
        
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'trainval': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }
        
        self.transform = data_transforms[split]
        self.word2vocab_id = json.load(open('word2vocab_id.json'))
        self.ans2id = json.load(open('ans2id.json'))
        
        dataroot='../data'
        dataset = json.load(open(os.path.join(dataroot, '%s_%s.json'%(dataset_name, split))), encoding='cp1252')
        max_length = 20
        for sample in dataset:
            question = sample['question']
            question = question.lower()
            tokens = nltk.word_tokenize(question)
            tokens = [self.word2vocab_id[x] for x in tokens if x in self.word2vocab_id]
            tokens = tokens[:max_length]
            if len(tokens) < max_length:
                # Note here we pad in front of the sentence
                padding = [0] * (max_length - len(tokens))
                tokens = padding + tokens
            sample['q_token'] = tokens
            
            tokens = [x.lower() for x in sample['answers']]
            tokens = [self.ans2id[x] for x in tokens if x in self.ans2id]
            sample['a_token'] = tokens
        self.dataset = dataset
        
        
    def __getitem__(self, index):
        entry = self.dataset[index]
        image = entry['image']
        image_path = os.path.join(self.image_dir, image.replace('.jpg', self.image_ext))
        with open(image_path, 'rb') as f:
            image = Image.open(f).convert('RGB')
        label = [0 if x < 2 else 1 for x in entry['ans_dis_labels']]
        label = torch.tensor(label, dtype=torch.float32)
        
        question = torch.from_numpy(np.array(entry['q_token']))
        answer = np.zeros((len(self.ans2id),), dtype=np.float32)
        for ans in entry['a_token']:
            answer[ans] += 1.0
        answer /= 10.0
        answer = torch.from_numpy(answer)
        
        if self.transform:
            image = self.transform(image)

        return image, question, answer, label
    
    def __len__(self):
        return len(self.dataset)
    

splits = ['train', 'val', 'test']
datasets = {}
datasets.update({x: AnswerDisDataset('vizwiz', x) for x in splits})
dataset_sizes = {x: len(datasets[x]) for x in splits}
print(dataset_sizes)

{'test': 7501, 'train': 19502, 'val': 3001}


In [3]:
from language_model import WordEmbedding, QuestionEmbedding
from fc import FCNet
from torch.nn.utils.weight_norm import weight_norm

class Identity(nn.Module):
    def __init__(self, dim):
        super(Identity, self).__init__()
        self.dim = dim
        
    def forward(self, x):
        return x

def init_image_model(init_model_path=None):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = Identity(num_ftrs)
    
    if init_model_path:
        model.load_state_dict(torch.load(init_model_path))
    model = model.to(device)
    return model

class BaseModel(nn.Module):
    def __init__(self, w_emb, q_emb, q_net, v_net, a_net, classifier, model_type='Q+I'):
        super(BaseModel, self).__init__()
        self.w_emb = w_emb
        self.q_emb = q_emb
        self.q_net = q_net
        self.v_net = v_net
        self.a_net = a_net
        self.classifier = classifier
        self.model_type = model_type

    def forward(self, v, q, a):
        """Forward

        return: logits, not probs
        """
        w_emb = self.w_emb(q)
        q_emb = self.q_emb(w_emb) # [batch, q_dim]

        q_repr = self.q_net(q_emb)
        v_repr = self.v_net(v)
        a_repr = self.a_net(a)
        
        model_type = self.model_type
        if model_type == 'I':
            joint_repr = v_repr
        elif model_type == 'Q':
            joint_repr = q_repr
        elif model_type == 'A':
            joint_repr = a_repr
        elif model_type == 'Q+I':
            joint_repr = q_repr * v_repr
        elif model_type == 'Q+I+A':
            joint_repr = (q_repr + v_repr + a_repr) / 3
        elif model_type == 'Q+A':
            joint_repr = q_repr * a_repr
        logits = self.classifier(joint_repr)
        return logits
    
class FullModel(nn.Module):
    def __init__(self, image_model, base_model):
        super(FullModel, self).__init__()
        self.image_model = image_model
        self.base_model = base_model
        
    def forward(self, img, q, a):
        x = self.image_model(img)
        x = self.base_model(x, q, a)
        return x

def init_model(model_type='Q+I'):
    num_hid = 300
    image_model = init_image_model()
    w_emb = WordEmbedding(len(datasets['train'].word2vocab_id), 300, 0.0)
    w_emb.init_embedding('glove6b_init_300d.npy')
    q_emb = QuestionEmbedding(300, num_hid, 1, False, 0.0)
    q_net = FCNet([num_hid, num_hid])
    v_net = FCNet([image_model.fc.dim, num_hid])
    a_net = FCNet([len(datasets['train'].ans2id), num_hid])
    classifier = weight_norm(nn.Linear(num_hid, 10), dim=None)

    base_model = BaseModel(w_emb, q_emb, q_net, v_net, a_net, classifier, model_type)
    base_model = base_model.to(device)
    model = FullModel(image_model, base_model)
    return model

In [4]:
def evaluate_model(model, dataloader):
    model.eval() 
    score_all = []
    label_all = []

    # Iterate over data.
    for images, questions, answers, labels in dataloader:
        images = images.to(device)
        questions = questions.to(device)
        labels = labels.to(device)
        answers = answers.to(device)
        outputs = model(images, questions, answers)
        score_all.append(outputs.data.cpu().numpy())
        label_all.append(labels.data.cpu().numpy())

    score_all = np.concatenate(score_all, axis=0)
    label_all = np.concatenate(label_all, axis=0)
    #score_all[np.isnan(score_all)] = 0.
    ap = average_precision_score(label_all, score_all, average=None)
    
    return ap, label_all, score_all

def train_model(model, num_epochs=5, train_splits=['train'], 
                eval_splits=['test'], n_epochs_per_eval = 1):
    criterion = nn.BCEWithLogitsLoss()
    params = [{'params': model.base_model.parameters()}]
    optimizer = optim.Adam(params, lr=1e-3)
    # Decay LR by a factor of 0.1 every 100 epochs
    scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_ap = 0.0
    
    
    train_dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4) for x in train_splits}
    
    eval_dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=32,
                                         shuffle=False, num_workers=4) for x in eval_splits}
    
    dataloaders = {}
    dataloaders.update(train_dataloaders)
    dataloaders.update(eval_dataloaders)
    
    ###########evaluate init model###########
    for eval_split in eval_splits:
        ap, label, score = evaluate_model(model, dataloaders[eval_split])
        print('(AP={1}) {0}'.format(eval_split, 100*ap))
        ap = np.mean(ap)
        print(100*ap)
    print()
    #########################################

    for epoch in range(num_epochs):
        since = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        scheduler.step()
        
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for train_split in train_splits:
            for images, questions, answers, labels in dataloaders[train_split]:
                model.train()  # Set model to training mode
                images = images.to(device)
                questions = questions.to(device)
                labels = labels.to(device)
                answers = answers.to(device)
                outputs = model(images, questions, answers)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # compute average precision
        if (epoch+1) % n_epochs_per_eval == 0:
            for eval_split in eval_splits:
                ap, label, score = evaluate_model(model, dataloaders[eval_split])
                print('(AP={1}) {0}'.format(eval_split, 100*ap))
                ap = np.mean(ap)
                print(100*ap)
            # deep copy the model
                
            if ap > best_ap:
                best_ap = ap
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Epoch time: {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        #print(flush=True)
    
    ###########evaluate final model###########
    for eval_split in eval_splits:
        ap, label, score = evaluate_model(model, dataloaders[eval_split])
        print('(AP={1}) {0}'.format(eval_split, 100*ap))
        ap = np.mean(ap)
        print(100*ap)
    # deep copy the model
    if ap > best_ap:
        best_ap = ap
        best_model_wts = copy.deepcopy(model.state_dict())
    print()
    #########################################

    print('Best val AP: {:2f}'.format(100*best_ap))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return

In [5]:
train_splits = ['train', 'val']
model_type_list = ['I', 'Q', 'Q+I', 'Q+A', 'A', 'Q+I+A']
eval_splits=['test']

for model_type in model_type_list:
    print(model_type)

    model = init_model(model_type)
    save_model_path = 'saved_models/{0}_train-on-({1}).pt'.format(model_type, ','.join(train_splits))
    train_model(model, num_epochs=5, train_splits=train_splits, eval_splits=eval_splits, n_epochs_per_eval = 1)
    torch.save(model.state_dict(), save_model_path)
    print('\n')

I
(AP=[24.32 31.84 18.28  4.92 78.63  6.   66.81 77.63  1.62  0.82]) test
31.08674423292475

Epoch 0/4
----------
(AP=[51.56 49.25 28.67  6.76 82.25  8.34 78.34 85.45  1.63  0.41]) test
39.2650905991656
Epoch time: 3m 54s
Epoch 1/4
----------
(AP=[53.17 49.99 29.77  8.17 82.85  8.85 79.15 85.8   1.79  0.48]) test
40.00175747234163
Epoch time: 3m 53s
Epoch 2/4
----------
(AP=[54.18 50.39 29.77  7.62 83.38  8.93 79.41 85.97  1.82  0.54]) test
40.200822311611724
Epoch time: 3m 55s
Epoch 3/4
----------
(AP=[54.83 50.38 29.96  8.77 83.32  9.   79.57 86.13  1.7   0.57]) test
40.42276496570785
Epoch time: 3m 55s
Epoch 4/4
----------
(AP=[55.42 50.66 30.12  8.77 83.39  8.64 79.76 86.29  1.71  0.61]) test
40.536735651667364
Epoch time: 3m 54s
(AP=[55.42 50.66 30.12  8.77 83.39  8.64 79.76 86.29  1.71  0.61]) test
40.536735651667364

Best val AP: 40.536736


Q
(AP=[22.14 34.64 16.67  5.15 72.43  6.06 64.03 71.99  1.49  0.45]) test
29.505171556363518

Epoch 0/4
----------
(AP=[35.53 53.78 37.    

In [6]:
model_type_list = ['A', 'I', 'Q', 'Q+I', 'Q+A', 'Q+I+A']
train_splits = ['train', 'val']
eval_splits=['test']

dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=32,
                                     shuffle=False, num_workers=4) for x in eval_splits}

score_all = 0
all_data= {}
for model_type in model_type_list:
    print(model_type)
    model = init_model(model_type)
    save_model_path = 'saved_models/{0}_train-on-({1}).pt'.format(model_type, ','.join(train_splits))
    model.load_state_dict(torch.load(save_model_path))
    
    eval_split = eval_splits[0]
    ap, label, score = evaluate_model(model, dataloaders[eval_split])
    
    key = '%s_train-on-%s_%s'%(model_type, ','.join(train_splits), eval_split)
    value = [ap, label, score]
    all_data[key] = value
    
    print('AP: {1}\nmAP: {2}'.format(eval_split, 100*ap, 100*np.mean(ap)))
    print('\n')
    
    score_all += score

A
AP: [65.39 77.8  57.48  9.83 89.08 10.22 90.08 95.15  1.75  2.27]
mAP: 49.904512048


I
AP: [55.42 50.66 30.12  8.77 83.39  8.64 79.76 86.29  1.71  0.61]
mAP: 40.5367356517


Q
AP: [35.87 54.66 39.24 12.32 84.41 11.   79.46 85.1   2.15  0.76]
mAP: 40.4975274849


Q+I
AP: [57.81 62.47 43.24 13.77 87.81 11.14 86.36 92.01  2.    0.75]
mAP: 45.7348860413


Q+A
AP: [65.28 77.85 57.41 10.55 88.79 10.1  90.1  95.22  1.77  1.66]
mAP: 49.8716331909


Q+I+A
AP: [66.25 77.71 57.2  13.55 90.01 12.46 90.53 95.51  1.96  1.62]
mAP: 50.6784709973




In [11]:
 np.mean([44.82, 58.63,18.15,  5.7,  80.14,  5.14, 66.61, 71.94,  1.35,  0.62])

35.31

In [7]:
score = [all_data['%s_train-on-%s_%s'%(model_type, ','.join(train_splits), eval_split)][2]
         for model_type in ['A', 'Q']]
score = sum(score)
ap = average_precision_score(label, score, average=None)
print('(AP={1}) {0}'.format(eval_split, 100*ap))
ap = np.mean(ap)
print(100*ap)

(AP=[61.28 76.64 56.05 11.94 89.65 12.09 90.26 94.92  2.04  1.09]) test
49.59655459581514


In [8]:
LQI	IVE	INV	DFF	AMB	SBJ	SYN	GRN	SPM	OTH
7.89 56.53 38.25 25.78 96.37 24.97 87.92 83.27  5.39  0.4

SyntaxError: invalid syntax (<ipython-input-8-6c81065af7b4>, line 1)